In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAI
from langchain_chroma import Chroma

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
import pandas as pd
books = pd.read_csv("books_cleaned.csv")
books.head(3)

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."


In [5]:
books["tagged_description"]

0       9780002005883 A NOVEL THAT READERS and critics...
1       9780002261982 A new 'Christie for Christmas' -...
2       9780006178736 A memorable, mesmerizing heroine...
3       9780006280897 Lewis' work on the nature of lov...
4       9780006280934 "In The Problem of Pain, C.S. Le...
                              ...                        
5192    9788172235222 On A Train Journey Home To North...
5193    9788173031014 This book tells the tale of a ma...
5194    9788179921623 Wisdom to Create a Life of Passi...
5195    9788185300535 This collection of the timeless ...
5196    9789027712059 Since the three volume edition o...
Name: tagged_description, Length: 5197, dtype: str

In [6]:
books["tagged_description"].to_csv(
    "tagged_description.txt",
    index=False,
    header=False
)

In [7]:
raw_documents = TextLoader("tagged_description.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1000 ,chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 1184, which is longer than the specified 1000
Created a chunk of size 1216, which is longer than the specified 1000
Created a chunk of size 1090, which is longer than the specified 1000
Created a chunk of size 1191, which is longer than the specified 1000
Created a chunk of size 1267, which is longer than the specified 1000
Created a chunk of size 2016, which is longer than the specified 1000
Created a chunk of size 1225, which is longer than the specified 1000
Created a chunk of size 1186, which is longer than the specified 1000
Created a chunk of size 1216, which is longer than the specified 1000
Created a chunk of size 1193, which is longer than the specified 1000
Created a chunk of size 1059, which is longer than the specified 1000
Created a chunk of size 1272, which is longer than the specified 1000
Created a chunk of size 1637, which is longer than the specified 1000
Created a chunk of size 1132, which is longer than the specified 1000
Created a chunk of s

In [8]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='"9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. Itâ€™s 1956 in Gilead, Iowa, towards the end of the Reverend Amesâ€™s life, and he is absorbed in recording his familyâ€™s story, a legacy for the young son he will never see grow up. Haunted by his grandfatherâ€™s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friendâ€™s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Amesâ€™s joyous, rambling voice that finds beauty, humour and truth in the smallest of lifeâ

In [9]:
import os
from langchain_openai import OpenAIEmbeddings
db_books = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings()
)

In [10]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k=10)
docs

[Document(id='7f9c67ec-9061-4a0d-a1ea-b1585a186a01', metadata={'source': 'tagged_description.txt'}, page_content='"9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience."\n"9780786808373 Introducing your baby to birds, cats, dogs, and babies through fine art, illsutration and photographs. These books are a rare opportunity to expose little ones to a range of images on a single subject, from simple child\'s drawings and abstract art to playful photos. A brief text accompanies each image, introducing baby to some basic -- and sometimes playful -- information on the subjects."'),
 Document(id='34896f75-923e-42ab-82dd-e74db171652b', metadata={'source': 'tagged_description.txt'}, page_content='"9780786808380 Introduce your babies to birds, cats, dogs, and babies through f

In [14]:
isbn = docs[0].page_content.split()[0].strip().replace('"', '')
books[books["isbn13"] == isbn]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
3747,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...


In [12]:
def retrieve_semantic_recommendations(query: str, top_k: int = 10) -> pd.DataFrame:
    # Perform semantic search
    recs = db_books.similarity_search(query, k=50)
    books_list = [
        rec.page_content.strip().split()[0].replace('"', '')
        for rec in recs
    ]
    books['isbn13'] = books['isbn13'].astype(str).str.strip()
    return books[books['isbn13'].isin(books_list)].head(top_k)

In [13]:
retrieve_semantic_recommendations("A book to teach children about nature")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
30,9780006646006,000664600X,Ocean Star Express,Mark Haddon;Peter Sutton,Juvenile Fiction,http://books.google.com/books/content?id=I2QZA...,Joe and his parents are enjoying a summer holi...,2002.0,3.50,32.0,1.0,Ocean Star Express,9780006646006 Joe and his parents are enjoying...
223,9780060775858,0060775858,Goodnight Moon 60th Anniversary Edition,Margaret Wise Brown,Juvenile Fiction,http://books.google.com/books/content?id=lLYOr...,"In a great green room, tucked away in bed, is ...",2005.0,4.27,32.0,264013.0,Goodnight Moon 60th Anniversary Edition,"9780060775858 In a great green room, tucked aw..."
260,9780060852559,0060852550,"Animal, Vegetable, Miracle",Barbara Kingsolver;Camille Kingsolver;Steven L...,Biography & Autobiography,http://books.google.com/books/content?id=qLkEY...,Bestselling author Barbara Kingsolver returns ...,2007.0,4.04,370.0,86130.0,"Animal, Vegetable, Miracle: A Year of Food Life",9780060852559 Bestselling author Barbara Kings...
324,9780060959036,0060959037,Prodigal Summer,Barbara Kingsolver,Fiction,http://books.google.com/books/content?id=06IwG...,Barbara Kingsolver's fifth novel is a hymn to ...,2001.0,4.00,444.0,85440.0,Prodigal Summer: A Novel,9780060959036 Barbara Kingsolver's fifth novel...
399,9780062516374,006251637X,Rainforest Home Remedies,Rosita Arvigo;Nadine Epstein,Health & Fitness,http://books.google.com/books/content?id=DLnwO...,Rainforest Healing from Your Home and Garden F...,2001.0,4.12,221.0,65.0,Rainforest Home Remedies: The Maya Way To Heal...,9780062516374 Rainforest Healing from Your Hom...
413,9780064405850,0064405850,Strawberry Girl 60th Anniversary Edition,Lois Lenski,Juvenile Fiction,http://books.google.com/books/content?id=AQXM2...,"The land was theirs, but so were its hardships...",1995.0,3.86,208.0,10655.0,Strawberry Girl 60th Anniversary Edition,"9780064405850 The land was theirs, but so were..."
424,9780064410724,0064410722,Four Stupid Cupids,Gregory Maguire,Juvenile Fiction,http://books.google.com/books/content?id=471OU...,The students' scheme to find a love match for ...,2001.0,3.52,224.0,110.0,Four Stupid Cupids,9780064410724 The students' scheme to find a l...
442,9780067575208,006757520X,The Sense of Wonder,Rachel Carson,Nature,http://books.google.com/books/content?id=Zee5S...,"First published more than three decades ago, t...",1998.0,4.39,112.0,1160.0,The Sense of Wonder,9780067575208 First published more than three ...
774,9780141309408,0141309407,Serena and the Sea Serpent,Garth Nix,Sea monsters,http://books.google.com/books/content?id=jb4vP...,Novel in the Aussie Bites series for early pri...,2000.0,3.59,79.0,33.0,Serena and the Sea Serpent,9780141309408 Novel in the Aussie Bites series...
799,9780142003343,0142003344,The Blank Slate,Steven Pinker,Psychology,http://books.google.com/books/content?id=7rJ5g...,In a study of the nature versus nurture debate...,2003.0,4.08,528.0,17851.0,The Blank Slate: The Modern Denial of Human Na...,9780142003343 In a study of the nature versus ...
